In [1]:
pip install splinter webdriver_manager

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo
import pathlib

In [2]:
# Path to chromedriver


/usr/local/bin/chromedriver


In [3]:
# Set the executable path and initialize the chrome browser in splinter


## Visit the NASA mars news site

In [4]:
# Visit the mars nasa news site

# Optional delay for loading the page


True

In [5]:
# Convert the browser html to a soup object and then quit the browser


In [7]:
# .find() the content title and save it as `news_title`


"How NASA's Mars Helicopter Will Reach the Red Planet's Surface"

In [8]:
# .find() the paragraph text


'The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.'

## JPL Space Images Featured Image

In [9]:
# Visit JPL space images Mars URL 


In [10]:
# Find and click the full image button


In [11]:
# Find the more info button and click that


In [12]:
# Parse the resulting html with soup


In [13]:
# find the relative image url


'/spaceimages/images/largesize/PIA20063_hires.jpg'

In [14]:
# Use the base url to create an absolute url


'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA20063_hires.jpg'

## Mars Facts

In [3]:
# Create a dataframe from the space-facts.com mars page

In [5]:
# clean the dataframe and export to HTML

## Hemispheres

In [ ]:
# visit the USGS astrogeology page for hemisphere data from Mars

In [19]:


# First, get a list of all of the hemispheres

# Next, loop through those links, click the link, find the sample anchor, return the href
    
    # We have to find the elements on each loop to avoid a stale element exception
    
    
    # Next, we find the Sample image anchor tag and extract the href
    
    
    # Get Hemisphere title
    
    
    # Append hemisphere object to list
    
    
    # Finally, we navigate backwards with browser.back()
    

In [20]:
# view the hemisphere urls to make sure they look good

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]